<a href="https://colab.research.google.com/github/LongB260/Product-Recognition-and-Order-Management-Telegram-Bot/blob/main/Product_Recognition_and_Order_Management_Telegram_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-telegram-bot torch torchvision ultralytics
!pip install pyTelegramBotAPI
!pip install pyngrok telebot ultralytics Pillow requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 652.1/652.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.3/263.3 kB 6.8 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO
# Load model
model = YOLO('/content/yolov8n.pt')
print(model.names)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 24.3MB/s]


{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [5]:
import telebot
import requests
import os
import signal
import sys
import shutil
import csv
from ultralytics import YOLO
from PIL import Image
from io import BytesIO

TOKEN = '8125275712:AAGKiYcW_UYSHJolV70b5vb0WqgBT_zX-lY'
bot = telebot.TeleBot(TOKEN)

DOWNLOAD_DIR = 'downloaded_images'
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

model = YOLO('yolov8n.pt')

# Menu items with images and prices
ITEMS = {
    "0": {"name": "frisbee", "image": 'https://th.bing.com/th/id/R.1111e44ee4d2b54eee7b5c2f1ff55ed3?rik=Hg7QP2OLTE7JzQ&pid=ImgRaw&r=0', "price": "220 000VND"},
    "1": {"name": "skis", "image": 'https://th.bing.com/th/id/OIP.f65x_EbJERsy6EnC5J4-JgHaHa?rs=1&pid=ImgDetMain', "price": "300 000VND"},
    "2": {"name": "snowboard", "image": 'https://th.bing.com/th/id/OIP.JIeyBeX3HO3j26BVlnrIjwHaJ4?rs=1&pid=ImgDetMain', "price": "250 000VND"},
    "3": {"name": "baseball glove", "image": 'https://cdn.shopify.com/s/files/1/1682/7299/products/PROS30390-6TN-1_1400x.png?v=1555427797', "price": "105 000VND"},
    "4": {"name": "skateboard", "image": 'https://th.bing.com/th/id/OIP.UgRiVYz_P-3GkqjgdaZWfAHaHa?rs=1&pid=ImgDetMain', "price": "200 000VND"}
}

# Dictionary to store addresses, orders, and user states
addresses = {}
orders = {}
user_states = {}

# Initialize CSV file for orders if it does not exist
if not os.path.exists("orders.csv"):
    with open("orders.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["User ID", "Address", "Item", "Quantity"])

# Function to send the list of available items
@bot.message_handler(commands=['list'])
def send_menu(message):
    menu_text = "Available items:\n" + "\n".join([f"{i}: {item['name']}" for i, item in ITEMS.items()])
    bot.reply_to(message, menu_text)

# Function to save the user's address
@bot.message_handler(func=lambda message: message.text and message.text.startswith("Address: "))
def save_address(message):
    user_id = message.from_user.id
    user_address = message.text.replace("Address: ", "")

    # Save address for the user
    addresses[user_id] = user_address
    bot.reply_to(message, f"Your address has been saved: {user_address}")

    # Ask what item they want to purchase
    send_menu(message)
    bot.reply_to(message, "Please select an item by typing its number.")
    user_states[user_id] = "selecting_item"

# Function to handle menu selection and ask for quantity
@bot.message_handler(func=lambda message: message.text.isdigit() and message.from_user.id in user_states and user_states[message.from_user.id] == "selecting_item")
def handle_menu_selection(message):
    user_id = message.from_user.id
    selected_item = ITEMS.get(message.text)

    if selected_item:
        orders[user_id] = {"item": selected_item['name']}
        bot.reply_to(message, f"You have selected {selected_item['name']}. Please enter the quantity.")
        user_states[user_id] = "entering_quantity"
    else:
        bot.reply_to(message, "Invalid item. Please select a valid item by typing its number.")

# Function to handle quantity and save the order to CSV
@bot.message_handler(func=lambda message: message.text.isdigit() and message.from_user.id in user_states and user_states[message.from_user.id] == "entering_quantity")
def save_order(message):
    user_id = message.from_user.id

    if user_id in orders:
        orders[user_id]['quantity'] = message.text
        user_address = addresses.get(user_id, "Unknown address")
        user_order = orders[user_id]

        # Save to CSV file
        with open("orders.csv", mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([user_id, user_address, user_order['item'], user_order['quantity']])

        # Confirm the order
        bot.reply_to(message, f"Your order has been saved!\nItem: {user_order['item']}\nQuantity: {user_order['quantity']}\nAddress: {user_address}")

        # Clean up the order for the user
        del orders[user_id]
        del user_states[user_id]
    else:
        bot.reply_to(message, "Please select an item first by typing its number.")

# Function to handle product identification from photos
@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    file_url = f'https://api.telegram.org/file/bot{TOKEN}/{file_info.file_path}'
    image_data = requests.get(file_url).content
    image = Image.open(BytesIO(image_data))
    temp_path = os.path.join(DOWNLOAD_DIR, "temp_image.jpg")
    image.save(temp_path)

    results = model(temp_path)

    # Check if results are found
    if results and results[0].boxes:
        best_label = max(results[0].boxes, key=lambda box: box.conf[0].item())
        detected_item = model.names[int(best_label.cls[0].item())].lower()

        # Find the matching item in the shop's product list
        for item in ITEMS.values():
            if detected_item == item['name']:
                bot.reply_to(message, f"Detected item: {item['name']}")
                send_item_info(message, item)
                return

    # If no matching item is found
    bot.reply_to(message, "No matching item found in the shop.")

    # Clean up image
    os.remove(temp_path)

# Function to send item information (image and price)
def send_item_info(message, item):
    bot.send_photo(message.chat.id, item["image"], caption=f"This is a {item['name']}. The price is {item['price']}.")

# Function to send the welcome message with options
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Welcome to the shop! You have two options:\n1. Type /list to view the available items.\n2. Type 'Address: [your address]' to enter your shipping address and start shopping.\n3. Send a photo to identify a product in the shop.")

# Clean up on exit
def cleanup_and_exit(signum, frame):
    shutil.rmtree(DOWNLOAD_DIR, ignore_errors=True)
    sys.exit(0)

signal.signal(signal.SIGINT, cleanup_and_exit)
signal.signal(signal.SIGTERM, cleanup_and_exit)

# Bot activation
bot.polling()




image 1/1 /content/downloaded_images/temp_image.jpg: 640x640 1 frisbee, 432.0ms
Speed: 21.1ms preprocess, 432.0ms inference, 32.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/downloaded_images/temp_image.jpg: 640x640 1 scissors, 225.1ms
Speed: 3.4ms preprocess, 225.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


SystemExit: 0

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
